## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.07,80,31,14.16,scattered clouds
1,1,Honiara,SB,-9.4333,159.9500,83.91,71,1,5.10,clear sky
2,2,Agutaya,PH,11.1480,120.9417,83.68,78,100,16.11,moderate rain
3,3,Kavieng,PG,-2.5744,150.7967,83.64,73,44,7.25,light rain
4,4,Hermanus,ZA,-34.4187,19.2345,53.91,83,11,6.82,few clouds
5,5,Tuktoyaktuk,CA,69.4541,-133.0374,60.80,88,90,10.36,overcast clouds
6,6,Port Macquarie,AU,-31.4333,152.9167,49.89,41,0,4.68,clear sky
7,7,Andenes,NO,69.3143,16.1194,49.98,87,83,23.02,broken clouds
8,8,Itarema,BR,-2.9248,-39.9167,78.73,84,76,12.62,broken clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.16,81,44,9.19,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.07,80,31,14.16,scattered clouds
1,1,Honiara,SB,-9.4333,159.9500,83.91,71,1,5.10,clear sky
2,2,Agutaya,PH,11.1480,120.9417,83.68,78,100,16.11,moderate rain
3,3,Kavieng,PG,-2.5744,150.7967,83.64,73,44,7.25,light rain
8,8,Itarema,BR,-2.9248,-39.9167,78.73,84,76,12.62,broken clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.16,81,44,9.19,scattered clouds
10,10,Albany,US,42.6001,-73.9662,70.66,98,100,6.13,overcast clouds
13,13,Alofi,NU,-19.0595,-169.9187,76.89,78,40,16.11,scattered clouds
15,15,Sao Luis,BR,-2.5297,-44.3028,79.00,83,75,5.75,moderate rain
18,18,Thompson,CA,55.7435,-97.8558,75.36,35,40,8.05,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                340
City                   340
Country                340
Lat                    340
Lng                    340
Max Temp               340
Humidity               340
Cloudiness             340
Wind Speed             340
Current Description    340
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.07,scattered clouds,-9.8000,-139.0333,
1,Honiara,SB,83.91,clear sky,-9.4333,159.9500,
2,Agutaya,PH,83.68,moderate rain,11.1480,120.9417,
3,Kavieng,PG,83.64,light rain,-2.5744,150.7967,
8,Itarema,BR,78.73,broken clouds,-2.9248,-39.9167,
9,Puerto Ayora,EC,75.16,scattered clouds,-0.7393,-90.3518,
10,Albany,US,70.66,overcast clouds,42.6001,-73.9662,
13,Alofi,NU,76.89,scattered clouds,-19.0595,-169.9187,
15,Sao Luis,BR,79.00,moderate rain,-2.5297,-44.3028,
18,Thompson,CA,75.36,scattered clouds,55.7435,-97.8558,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.07,scattered clouds,-9.8000,-139.0333,Villa Enata
1,Honiara,SB,83.91,clear sky,-9.4333,159.9500,Heritage Park Hotel
3,Kavieng,PG,83.64,light rain,-2.5744,150.7967,Nusa Island Retreat
8,Itarema,BR,78.73,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
9,Puerto Ayora,EC,75.16,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
667,Forio,IT,83.82,clear sky,40.7342,13.8646,Hotel Terme Providence
668,Baghdad,IQ,93.11,clear sky,33.3406,44.4009,Baghdad Hotel
672,Ormara,PK,81.72,overcast clouds,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
673,Agropoli,IT,84.52,clear sky,40.3469,14.9966,B&B Baia di Trentova


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))